<a href="https://colab.research.google.com/github/anatorres09/DE-MRI_Classificacio_CNN/blob/main/model.CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Connectar a Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
#Importar llibreries
import os
import cv2
import numpy as np
from glob import glob
import tensorflow as tf

In [ ]:
# Carregar i preprocés d'imatges per a la predicció
ruta_carpeta = '/content/drive/MyDrive/CNN'

# Ruta del model
ruta_model = os.path.join(ruta_carpeta, "model_CNN.h5")

# Carregar el model
model = tf.keras.models.load_model(ruta_model)

ruta_carpeta_test = os.path.join(ruta_carpeta, "test")
redimensionar_imatge = (128, 128)

# Funció per a recollir les imatges
def recollir_imatges(ruta_carpeta, redimensionar_imatge):
    casos = [nom_carpeta for nom_carpeta in os.listdir(ruta_carpeta) if os.path.isdir(os.path.join(ruta_carpeta, nom_carpeta))]

    imatges = []
    noms_fitxers = []

    for cas in casos:
        ruta_cas = os.path.join(ruta_carpeta, cas)
        rutes_imatges = glob(os.path.join(ruta_cas, '*.png'))

        for ruta_imatge in rutes_imatges:
            imatge = cv2.imread(ruta_imatge)
            imatge = cv2.cvtColor(imatge, cv2.COLOR_BGR2RGB)
            
            # Redimensionar les imatges
            imatge = cv2.resize(imatge, redimensionar_imatge)
            
            imatges.append(imatge)
            noms_fitxers.append(ruta_imatge)

    return imatges, noms_fitxers



# Recollir imatges de la carpeta de prova
imatges_test, noms_fitxers = recollir_imatges(ruta_carpeta_test, redimensionar_imatge)

# Convertir la llista d'imatges en una matriu numpy
matriu_imatges_test = np.array(imatges_test, dtype='float32')

print(f"S'han recollit {matriu_imatges_test.shape[0]} imatges en total.")

# Realitzar la predicció utilitzant el model carregat
predicciones = model.predict(matriu_imatges_test)

# Obtindre les classes predites ('normal' o 'patologic')
classes_predites = np.argmax(predicciones, axis=1)

# Obtindre les probabilitats màximes
probabilidades_maximas = np.max(predicciones, axis=1)

# Calcular umbral dinàmic en funció de la probabilitat màxima
umbral = np.mean(probabilidades_maximas)

# Imprimir les classes predites per a cada imatge en la carpeta de prova
for i in range(matriu_imatges_test.shape[0]):
    nom_fitxer = os.path.basename(noms_fitxers[i])
    classe_predita = classes_predites[i]
   
    if classe_predita == 0:
        classe = 'normal'
    else:
        classe = 'patologic'
    
    print(f"La classe predita per a la imatge {nom_fitxer} és: {classe}")

